# Locqueneux Timéo, Le Rossignol Pierre-Jean, Bodin Titouan, Vandemeulebroucke Sacha

In [29]:
%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from IPython.display import Image

# INITIALISATION

In [31]:
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

walletAlice = Wallet()
walletSacha = Wallet()
walletTiméo = Wallet()


pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)




# Smart contract importation

In [32]:
from smart_contract import SmartContractDefinition, SmartContractWritingOperation
from CollectionNFT import Collection
from certificate import Certificate
import inspect


reset_blockchain()


# collection : string du smart contract
collection = inspect.getsource(Collection)
dummySmartContractDefinition = SmartContractDefinition(walletAlice.publicKey, collection)
walletAlice.sign(dummySmartContractDefinition)

collection_nft = dummySmartContractDefinition.instantiate_collection(8)
node1.new_certificate(dummySmartContractDefinition)





Users can “mint” tokens under certain conditions :  
- Once the limit of N has been reached, there is no minting allowed anymore. 
- There should be a “mint” period. The owner of the collection can open the collection

In [33]:
open_minting_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'open_minting',
    [node1.blockchain.get_latest_block().indexInBlockchain,2]
)
walletAlice.sign(open_minting_SmartContractWritingOperation)
open_minting_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(open_minting_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

    

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    [node1.blockchain.get_latest_block().indexInBlockchain]
)
walletAlice.sign(mint_token_SmartContractWritingOperation)
mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(mint_token_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

Minting is open
Minted !
ownership changed


# Mint period with number of blocks

In [34]:

for i in range(15):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
walletAlice.publicKey,
dummySmartContractDefinition.hash(),
'mint_token',
[node1.blockchain.get_latest_block().indexInBlockchain]
)
walletAlice.sign(mint_token_SmartContractWritingOperation)
mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(mint_token_SmartContractWritingOperation)


Minting is closed


# Limit of N tokens

In [35]:
open_minting_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'open_minting',
    [node1.blockchain.get_latest_block().indexInBlockchain,10]
)
walletAlice.sign(open_minting_SmartContractWritingOperation)
open_minting_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(open_minting_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)



    

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    [node1.blockchain.get_latest_block().indexInBlockchain]
)

for i in range (10):
    walletAlice.sign(mint_token_SmartContractWritingOperation)
    mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
    node1.new_certificate(mint_token_SmartContractWritingOperation)


    for i in range(4):
        certificate = Certificate(walletAlice.publicKey)
        walletAlice.sign(certificate)
        node1.new_certificate(certificate)


Minting is open
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Minted !
ownership changed
Limit of tokens reached
ownership changed
Limit of tokens reached
ownership changed
Limit of tokens reached
ownership changed


# Users can exchange ownership of their tokens

In [36]:
change_ownership_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'change_ownership',
    [1,walletTiméo.publicKey]
)

walletAlice.sign(change_ownership_SmartContractWritingOperation)
change_ownership_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(change_ownership_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

ownership changed


In [37]:
def get_token_owner(token_number, smart_contract, node):
    smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node.blockchain,
    smart_contract.hash())
    print('\n','token number ', token_number, ' has public key :', smartContractObject.tokens[f'Token No{token_number}']['owner'][256:272])

get_token_owner(1, dummySmartContractDefinition, node1)
print("Timéo's wallet :      ",walletTiméo.publicKey[256:272])

change_ownership_SmartContractWritingOperation = SmartContractWritingOperation(
    walletTiméo.publicKey,
    dummySmartContractDefinition.hash(),
    'change_ownership',
    [1,walletSacha.publicKey]
)
walletTiméo.sign(change_ownership_SmartContractWritingOperation)
change_ownership_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(change_ownership_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)


get_token_owner(1, dummySmartContractDefinition, node1)
print("Sacha's wallet :    ",walletSacha.publicKey[256:272])

Minting is open
Minted !
ownership changed
Minting is closed
Minting is open
Minted !
ownership changed
ownership changed

 token number  1  has public key : dd519d2b71428af2
Timéo's wallet :       dd519d2b71428af2
ownership changed
Minting is open
Minted !
ownership changed
Minting is closed
Minting is open
Minted !
ownership changed
ownership changed
ownership changed

 token number  1  has public key : 228ee435d7343ca6
Sacha's wallet :     228ee435d7343ca6


# Create a collection of N tokens that all have a distinguishable identifier

In [38]:
print(collection_nft.tokens)
for token in collection_nft.tokens:
    print(token)

{'Token No0': {'name': 'pikachu', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png', 'description': 'electric'}, 'Token No1': {'name': 'bulbasaur', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png', 'description': 'grass', 'owner': '30820122300d06092a864886f70d01010105000382010f003082010a02820101009bf0e3dba24cd106b59f045f57e2b8b083404d3cddc11a50a71738f650f3f782e25ab1e6754bee03d0505f489f1b5fb3471702b01ae90129c71d75e827b35fe5f5eb1115fcb9d8be92c4f8a6bbe417942b4876a7cd8f9fe898fb0705c5826e228ee435d7343ca6ac77ccbf05f378c0c9d92f1d0bd5cac1f64ddd652e69ebb94844d259fb2e1b852b6cd8af78ca8f6e2575473648dc77a2384debe44d009a939f7fc7073648b43b492abfc48c56e4cc518dfb94519257a1027c19fe4793e6c3ba5610a1bf3b3a05295db692e4925ed6881292744f3d9670993b9163ea4cb423bbb3457a111e6010ebfd0b62ad79105417801cbe8f432f629cb520719c62fade150203010001'}, 'Token No2': {'name': 'charmander', 'image_link': 'https://raw.githubusercontent.com

Each token must be linked with a unique image or a description of a game item

In [39]:
collection_nft.tokens['Token No3']

{'name': 'squirtle',
 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png',
 'description': 'water',
 'owner': '30820122300d06092a864886f70d01010105000382010f003082010a028201010090553389d69c622cba73c790b6acbb9ac5cb93be8f135e5ff632323e3ffdbe6d76e0b91d651eed825e3813f853eb63d9d38f772a7e87264987d373ea969aa698974e687ef1dba1011a72c13cdc43ed9483be2bd70050b1b85f9ebd2c047e0de8c41d443599b680fd16c4c7a8ad16dabb41d35e9628607e4b283b96197a886b04d632145f8509c98c2c2ff1c9f694771337e61cddc0de67a28b492b49929ef0322b8400feb6270747ed8ebb9cc8daab01b50712c1996ae7c812516404630220ec6874e1826eeece948a248e66144cfc46ef1451752b9677c92e744eb3105e3e5c4a82fba68fe59a448ff548e7d22061751cd6f40ed9540616c9b6aac135fd0ca70203010001'}

In [40]:
display(Image(url=collection_nft.tokens['Token No3']['image_link']))
display(Image(url=collection_nft.tokens['Token No1']['image_link']))
display(Image(url=collection_nft.tokens['Token No2']['image_link']))
display(Image(url=collection_nft.tokens['Token No4']['image_link']))
